### Configuration
#### Load dependencies

In [1]:
import cv2
import os
import face_recognition
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid

# import tf dependencies
import tensorflow as tf
from keras.models import Model
from keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten



2023-07-20 22:30:20.234749: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 22:30:20.331740: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 22:30:20.333426: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 22:30:21.192060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Config GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

#### Define constants

In [3]:
POSITIVE_PATH = os.path.normpath(os.path.join('data', 'positive'))
NEGATIVE_PATH = os.path.normpath(os.path.join('data', 'negative'))
ANCHOR_PATH = os.path.normpath(os.path.join('data', 'anchor'))

#### Create directories

In [4]:
os.makedirs(POSITIVE_PATH, exist_ok=True)
os.makedirs(NEGATIVE_PATH, exist_ok=True)
os.makedirs(ANCHOR_PATH, exist_ok=True)


### Data collection

In [51]:
cap = cv2.VideoCapture(0)
face_locations = None
capture_mode = None

while cap.isOpened():
    ret, frame = cap.read()
    if face_locations is None:
        face_locations = face_recognition.face_locations(frame)
        t, r, b, l = face_locations[0]
        y_margin = 250 - abs(t-b)
        x_margin = 250 - abs(l-r)
     # get 250 x 250 image basing on initial face position
    frame = frame[t-y_margin//2: b+y_margin//2, l - x_margin//2:r+x_margin//2, :]

    key = cv2.waitKey(1) & 0XFF
    # Change mode
    if key == ord('a'):
        capture_mode = 'a'
    elif key == ord('p'):
        capture_mode = 'p'
    elif key == ord('q'):
        break

    # collect samples based on current mode
    if capture_mode == 'a':
        cv2.imwrite(os.path.join(ANCHOR_PATH, f'{uuid.uuid1()}.jpg'), frame)
    elif capture_mode == 'p':
        cv2.imwrite(os.path.join(POSITIVE_PATH, f'{uuid.uuid1()}.jpg'), frame)

    cv2.imshow('Image collection', frame)

cap.release()
cv2.destroyAllWindows()

### Image preprocessing

In [10]:
anchor = tf.data.Dataset.list_files(os.path.join(ANCHOR_PATH, '*.jpg'))
positive = tf.data.Dataset.list_files(os.path.join(POSITIVE_PATH, '*.jpg'))
negative = tf.data.Dataset.list_files(os.path.join(NEGATIVE_PATH, '*.jpg'))

In [53]:
def preprocess(path_to_file):
    """Function that read file from file, resize and rescale numpy array"""
    img = tf.io.decode_jpeg(tf.io.read_file(path_to_file))
    return tf.image.resize(img, (100, 100)) / 255

def preprocess_twin(input_img, val_img, label):
    return preprocess(input_img), preprocess(val_img), label

In [52]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [54]:
data = data.map(preprocess_twin)
data = data.cache()
data.shuffle(buffer_size=1024)

<_ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [59]:
TRAIN_RATIO = 0.7
TEST_RATIO = 1 - TRAIN_RATIO

In [ ]:
test_data = data.skip(round(len(data)*TRAIN_RATIO))
test_data = test_data.take(round(len(data)*TEST_RATIO))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)